# Auto ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [2]:
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.widgets import RunDetails
import sklearn

import sys, os

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [3]:
# Worspace and experiment

ws = Workspace.from_config()
experiment_name = 'hyperdrive-udacity-capstone-automl'


experiment=Experiment(ws, experiment_name)

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

Workspace name: aml-wwe-ictx-dsplay
Azure region: westeurope
Subscription id: ec5ba19e-6205-418f-a52d-d0943090ca16
Resource group: rg-wwe-ictx-dsplayground


In [4]:
# Dataset


dataset_name = "HRAnalytics_train_dataset"

dataset = ws.datasets[dataset_name]

df = dataset.to_pandas_dataframe()
df.head()

,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,target
0,city_16,0.91,Male,Has relevent experience,no_enrollment,Masters,STEM,16,50-99,None,never,4,no
1,city_67,0.855,Male,Has relevent experience,Full time course,Masters,Other,16,None,None,>4,45,yes
2,city_21,0.624,Male,Has relevent experience,no_enrollment,Graduate,STEM,1,None,Pvt Ltd,never,68,yes
3,city_99,0.915,None,No relevent experience,Full time course,High School,None,10,10000+,Pvt Ltd,1,97,no
4,city_103,0.92,Male,Has relevent experience,no_enrollment,Graduate,STEM,6,None,Public Sector,2,116,yes


## Environment & Run Configuration





In [4]:
# Create compute cluster

compute_cluster_name = "alpha"

try:
    compute_cluster = ComputeTarget(workspace=ws, name=compute_cluster_name)
    print("Found existing cluster, please use it.")
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    compute_cluster = ComputeTarget.create(ws, compute_cluster_name, compute_config)

compute_cluster.wait_for_completion(show_output=True)

Found existing cluster, please use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [5]:
from azureml.core.runconfig import RunConfiguration

from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies
env = Environment('dataprep')

conda = CondaDependencies()
conda.add_conda_package("python=3.8")

# add pip packages
conda.add_pip_package('azureml-core')
conda.add_pip_package('pandas')
conda.add_pip_package('numpy')
conda.add_pip_package("scikit-learn==0.22.2.post1")

# add conda dependencies to environment
env.python.conda_dependencies = conda

runconfig = RunConfiguration()
runconfig.environment  = env
runconfig.target = compute_cluster





# AutoMl

- define a training job
- configure hyperdrive on that job


In [7]:
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 4,
    "n_cross_validations": 4,
    "primary_metric" : 'AUC_weighted'
}

automl_config = AutoMLConfig(compute_target=compute_cluster,
                             task="classification",
                             label_column_name="target",   
                             training_data =dataset,
                             enable_early_stopping=True,
                             featurization='auto',
                             debug_log="automl_errors.log",
                             **automl_settings
                            )


In [8]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [9]:
RunDetails(remote_run).show()
remote_run.wait_for_completion(show_output=False)


_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

{'runId': 'AutoML_fc1cd03b-c70e-4838-89fb-980255d11dd2',
 'target': 'alpha',
 'status': 'Completed',
 'startTimeUtc': '2021-09-08T14:00:07.040932Z',
 'endTimeUtc': '2021-09-08T14:29:06.543148Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'AUC_weighted',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '4',
  'target': 'alpha',
  'AMLSettingsJsonString': '{"path":null,"name":"hyperdrive-udacity-capstone-hyperdrive","subscription_id":"ec5ba19e-6205-418f-a52d-d0943090ca16","resource_group":"rg-wwe-ictx-dsplayground","workspace_name":"aml-wwe-ictx-dsplay","region":"westeurope","compute_target":"alpha","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"enable_batch_run":false,"enable_run_restructure":false,"start_auxiliary_runs_before_parent_complete":false,"enable_code_generation":false,"iterations":1000,"primary_metric":"AUC_we

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [5]:
from azureml.train.automl.run import AutoMLRun
remote_run = AutoMLRun(experiment, run_id="AutoML_fc1cd03b-c70e-4838-89fb-980255d11dd2")
best_run_automl, best_model_automl = remote_run.get_output()
best_run_metrics_automl = best_run_automl.get_metrics()


Package:azureml-automl-runtime, training version:1.33.1, current version:1.28.0.post2
Package:azureml-core, training version:1.33.0.post1, current version:1.28.0
Package:azureml-dataprep, training version:2.20.1, current version:2.15.0
Package:azureml-dataprep-native, training version:38.0.0, current version:33.0.0
Package:azureml-dataprep-rslex, training version:1.18.2, current version:1.13.0
Package:azureml-dataset-runtime, training version:1.33.0, current version:1.28.0
Package:azureml-defaults, training version:1.33.0, current version:1.28.0
Package:azureml-interpret, training version:1.33.0, current version:1.28.0
Package:azureml-mlflow, training version:1.33.0, current version:1.28.0
Package:azureml-pipeline-core, training version:1.33.0, current version:1.28.0
Package:azureml-telemetry, training version:1.33.0, current version:1.28.0
Package:azureml-train-automl-client, training version:1.33.0, current version:1.28.0
Package:azureml-train-automl-runtime, training version:1.33.1.

# Test Model

In [6]:

#dataset = Dataset.get_by_name(ws, name='HRAnalytics_test_dastest')
dataset = ws.datasets["HRAnalytics_test_dastest"]



In [7]:
test_df = dataset.to_pandas_dataframe()


In [8]:
y_true = test_df["target"]
x = test_df.drop(columns=["target"])

y_pred= best_model_automl.predict(x)


In [9]:
from sklearn.metrics import classification_report, confusion_matrix

print(classification_report(y_true, y_pred))

print(confusion_matrix(y_true, y_pred))

              precision    recall  f1-score   support

          no       0.85      0.88      0.87      2877
         yes       0.61      0.54      0.57       955

    accuracy                           0.80      3832
   macro avg       0.73      0.71      0.72      3832
weighted avg       0.79      0.80      0.80      3832

[[2546  331]
 [ 438  517]]


In [10]:
best_model_automl

PipelineWithYTransformations(Pipeline={'memory': None,
                                       'steps': [('datatransformer',
                                                  DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=False, observer=None, task='classification', working_dir='/mn...
)), ('extratreesclassifier', ExtraTreesClassifier(bootstrap=False, ccp_alpha=0.0, class_weight='balanced', criterion='gini', max_depth=None, max_features=None, max_leaf_nodes=None, max_samples=None, min_impurity_decrease=0.0, min_impurity_split=None, min_samples_leaf=0.01, min_samples_split=0.056842105263157895, min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=1, oob_score=False, random_state=None, verbose=0, warm_start=False))], verbose=False))], flatten_transform=None, weights=[0.06666666666666667, 0.3333333333333333, 0.066666666

In [12]:
best_run_automl.register_model("automlcapstone", 'outputs/model.pkl')

Model(workspace=Workspace.create(name='aml-wwe-ictx-dsplay', subscription_id='ec5ba19e-6205-418f-a52d-d0943090ca16', resource_group='rg-wwe-ictx-dsplayground'), name=automlcapstone, id=automlcapstone:1, version=1, tags={}, properties={})